In [15]:
import cv2
import numpy as np
import cvzone
import math
import mediapipe as mp
import time

import cv2
import numpy as np
import scipy.signal as signal
import scipy.fftpack as fftpack


caminput =0

In [16]:
##mediapipe hand detection
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose


In [17]:
cap = cv2.VideoCapture(caminput)
StartCoords = []
pose = mp.solutions.pose.Pose(model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5)
hand = mp.solutions.hands.Hands(max_num_hands=2,
    min_detection_confidence=0.5)
prev_time = 0
new_time = 0
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = cv2.flip(img, 1)
        img.flags.writeable = False
        height, width, _ = img.shape


        res = holistic.process(img)
        pose_res = pose.process(img)
        hand_res = hand.process(img)
        img.flags.writeable = True
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        for idx, landmark in enumerate(res.face_landmarks.landmark):
                    # Convert normalized coordinates to pixel values
                    x = int(landmark.x * width)
                    y = int(landmark.y * height)

                    # Draw a small circle at each landmark.
                    cv2.circle(img, (x, y), 1, (0, 255, 0), -1)

                    # Write the index number next to the landmark
                    cv2.putText(img, str(idx), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1)

        # if res.face_landmarks:
        #   face_landmarks = res.face_landmarks
        #   top_left_index = 116  # Example index for top-left corner
        #   top_right_index = 101  # Example index for top-right corner
        #   bottom_left_index = 192  # Example index for bottom-left corner
        #   bottom_right_index = 206  # Example index for bottom-right corner

        #   # Extracting coordinates
        #   top_left = face_landmarks.landmark[top_left_index]
        #   top_right = face_landmarks.landmark[top_right_index]
        #   bottom_left = face_landmarks.landmark[bottom_left_index]
        #   bottom_right = face_landmarks.landmark[bottom_right_index]

        #   # Convert normalized coordinates to pixel values
        #   points = [(int(pt.x * width), int(pt.y * height)) for pt in [top_left, top_right, bottom_left, bottom_right]]

        #   left_cheek = img[
        #       points[0][1]:points[3][1],
        #       points[0][0]:points[3][0]
        #   ]

        #   cv2.imshow("l", left_cheek)

        #   # Draw rectangle
        #   cv2.rectangle(img, points[0], points[3], (0, 255, 0), 2)

        # if res.face_landmarks:
        #   face_landmarks = res.face_landmarks
        #   top_left_index = 348  # Example index for top-left corner
        #   top_right_index = 101  # Example index for top-right corner
        #   bottom_left_index = 192  # Example index for bottom-left corner
        #   bottom_right_index = 433  # Example index for bottom-right corner

        #   # Extracting coordinates
        #   top_left = face_landmarks.landmark[top_left_index]
        #   top_right = face_landmarks.landmark[top_right_index]
        #   bottom_left = face_landmarks.landmark[bottom_left_index]
        #   bottom_right = face_landmarks.landmark[bottom_right_index]

        #   # Convert normalized coordinates to pixel values
        #   points = [(int(pt.x * width), int(pt.y * height)) for pt in [top_left, top_right, bottom_left, bottom_right]]

        #   points = [(int(pt.x * width), int(pt.y * height)) for pt in [top_left, top_right, bottom_left, bottom_right]]

        #   right_cheek = img[
        #       points[0][1]:points[3][1],
        #       points[0][0]:points[3][0]
        #   ]

        #   cv2.imshow("r", right_cheek)

        #   # Draw rectangle
        #   cv2.rectangle(img, points[0], points[3], (0, 255, 0), 2)



        cv2.imshow("Hand Tracking", img)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()


AttributeError: 'NoneType' object has no attribute 'landmark'

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
import time
cap = cv2.VideoCapture(0)
fps = int(cap.get(cv2.CAP_PROP_FPS))
vid_data=[]
curr_time=time.time()
while cap.isOpened():
    ret,frame=cap.read()
    if time.time()-curr_time>10:
        break
    frame=cv2.flip(frame,1)
    if ret is True:
        cv2.imshow('frame',frame)
        vid_data.append(frame)
        k=cv2.waitKey(10)
        if k ==ord("q"):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
vid_data=np.array(vid_data)

In [ ]:
# build gaussian pyramid for video
def gaussian_video(video_tensor,levels=3):
    for i in range(0,video_tensor.shape[0]):
        frame=video_tensor[i]
        pyr=build_gaussian_pyramid(frame,level=levels)
        gaussian_frame=pyr[-1]
        if i==0:
            vid_data=np.zeros((video_tensor.shape[0],gaussian_frame.shape[0],gaussian_frame.shape[1],3))
        vid_data[i]=gaussian_frame
    return vid_data

def temporal_ideal_filter(tensor,low,high,fps,axis=0):
    fft=fftpack.fft(tensor,axis=axis)
    frequencies = fftpack.fftfreq(tensor.shape[0], d=1.0 / fps)
    bound_low = (np.abs(frequencies - low)).argmin()
    bound_high = (np.abs(frequencies - high)).argmin()
    fft[:bound_low] = 0
    fft[bound_high:-bound_high] = 0
    fft[-bound_low:] = 0
    iff=fftpack.ifft(fft, axis=axis)
    return np.abs(iff)

def magnify_color(video_name,low,high,levels=3,amplification=50):
    # t,f=load_video(video_name)
    t=vid_data
    f=fps
    print(f)
    gau_video=gaussian_video(t,levels=levels)
    filtered_tensor=temporal_ideal_filter(gau_video,low,high,f)
    amplified_video=amplify_video(filtered_tensor,amplification=amplification)
    final=reconstract_video(amplified_video,t,levels=levels)
    print(final)
    return final